# Palabras Relacionadas - Dataset

En este notebook se explica en detalle el dataset y el procesamiento que requirió para dejarlo listo para la siguiente etapa del proyecto.

En primer lugar, se buscó y seleccionó múltiples textos en formato PDF pertenencientes a distintas materias de la carrera Ingeniería en Sistemas de Información, Universidad Tecnológica Nacional Facultad Regional Mendoza.

Estos, fueron cargados en la carpeta /data/raw.

Se siguió como convención para los nombres el número del año de la materia, seguido de un guión, una abreviatura del nombre de la materia, otro guión y el nombre original del material.

Utilizando algunas librerías de python, se convirtió cada archivo PDF en un archivo txt con su contenido, en /data/plain

In [ ]:
!pip install pypdf

In [2]:
from pypdf import PdfReader
import os
from os import listdir
from os.path import isfile, join

os.chdir('C:\\Users\\saoga\\OneDrive\\Escritorio\\Repos\\TPI-RNP-Palabras-Relacionadas-ISI')

In [5]:
raw_path = "./data/raw"
plain_path = "./data/plain"

# Arreglo con los nombres de los archivos PDF
raw_files = [os.path.splitext(f)[0] for f in listdir(raw_path) if isfile(join(raw_path, f))]

In [ ]:
metrics = {}
for f in raw_files:
    print("\033[94mConvirtiendo archivo: " + f + "\033[0m")

    sf = f.split("-")
    anio = sf[0].strip()
    materia = sf[1].strip()
    if (not anio in metrics):
        metrics[anio] = {}

    if (not materia in metrics[anio]):
        metrics[anio][materia] = 0

    metrics[anio][materia] += os.path.getsize(raw_path + "/" + f + ".pdf")

    reader = PdfReader(raw_path + "/" + f + ".pdf")
    for p in reader.pages:
        with open(plain_path + "/" + f + ".txt", "ab") as t:
            t.write(p.extract_text(
                extraction_mode="plain",
                layout_mode_space_vertically=False).encode("utf-8"))

print("\033[92m")
for a, materias in metrics.items():
    print("Año " + a + ":")
    for m, tamano in materias.items():
        print("\tMateria: " + m + " - " + str(round(tamano/1000000,2)) +"MB")
print("\033[0m")

Convirtiendo archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Convirtiendo archivo: 1 - AyED - cpp según yo pero en pedo
Convirtiendo archivo: 1 - AyED - cpp según yo
Convirtiendo archivo: 1 - AyED - Unidad3 (7929)
Convirtiendo archivo: 1 - AyED - Unidad4 (7930)
Convirtiendo archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Convirtiendo archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Convirtiendo archivo: 1 - SyO - 01 Evolucion de las estructuras
Convirtiendo archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Convirtiendo archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Convirtiendo archivo: 1 - SyO - 04 gestion-por-procesos
Convirtiendo archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Convirtiendo archivo: 1 - SyO - 1)La Información en la Empresa
Convirtiendo archivo: 1 - SyO - 2)Recopilacion de la informacion
Convirtiendo archivo: 1 - SyO - Analisis FODA
Con

A continuación, se tokenizó los archivos planos txt, generando un archivo txt nuevo donde cada línea es un token.

Esto se logró gracias a Spacy.

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [7]:
import spacy

tokens_path = "./data/tokens"

esp = spacy.load("es_core_news_sm")

In [ ]:
def is_clean_token(token):
    return not (
        token.is_punct or
        token.is_space or
        token.is_stop or
        len(token.text) == 1)

for f in raw_files:
    print("\033[94mTokenizando archivo: " + f + "\033[0m")

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = esp.tokenizer(txt)
        with open(tokens_path + "/" + f + ".txt", "wb") as tf:
            for token in tokens:
                if (is_clean_token(token)):
                    tf.write((token.text + "\n").encode("utf-8"))

Tokenizando archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Tokenizando archivo: 1 - AyED - cpp según yo pero en pedo
Tokenizando archivo: 1 - AyED - cpp según yo
Tokenizando archivo: 1 - AyED - Unidad3 (7929)
Tokenizando archivo: 1 - AyED - Unidad4 (7930)
Tokenizando archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Tokenizando archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Tokenizando archivo: 1 - SyO - 01 Evolucion de las estructuras
Tokenizando archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Tokenizando archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Tokenizando archivo: 1 - SyO - 04 gestion-por-procesos
Tokenizando archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Tokenizando archivo: 1 - SyO - 1)La Información en la Empresa
Tokenizando archivo: 1 - SyO - 2)Recopilacion de la informacion
Tokenizando archivo: 1 - SyO - Analisis FODA
Tokenizando archiv

Una vez que se tienen los archivos con los tokens, deseamos detectar conceptos dentro de ellos. Estos conceptos formarán luego nuestro vocabulario.

Se utilizará un algoritmo de ventana deslizante para esto.

In [9]:
from itertools import combinations
import re

window_size = 4

In [ ]:
related_table = {}

banned = ["capítulo", "página", "figura", "cap", "ejemplo", "catedra", "mendoza", "argentina", "muñoz", "facchini", "cesari", "xsd", "infoleg"]

for f in raw_files:
    print("\033[94mDetectando conceptos en archivo: " + f + "\033[0m")

    tokens = []

    with open(tokens_path + "/" + f + ".txt", "rb") as tf:
        tokens = tf.read().decode("utf-8").split("\n")

    for i in range(len(tokens) - window_size):
        window = tokens[i:i+window_size]

        def get_subarrays(arr):
            result = []
            n = len(arr)
            for r in range(1, n+1):  # sizes from 1 to n
                for indices in combinations(range(n), r):
                    subarray = [arr[i] for i in indices]
                    result.append(subarray)
            return result
        
        arrays = get_subarrays(window[1:])

        # arrays.insert(0, []) # Permite formar conceptos de una sola palabra

        for arr in arrays:
            arr.insert(0, window[0])
            arr = [s.lower() for s in arr]

            if any(re.search(r'(^[0-9\.\,]+$)|(-$)|(^.\.$)', s) for s in arr):
                continue

            if len(arr) != len(set(arr)):
                continue

            if any(ban in arr for ban in banned):
                continue

            arr.sort()
            
            t = tuple(arr)
            if (not t in related_table):
                related_table[t] = 0
            related_table[t] += 1

print("\033[92mCantidad de conceptos candidatos:" + str(len(related_table)) + "\033[0m")

Detectando conceptos en archivo: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
Detectando conceptos en archivo: 1 - AyED - cpp según yo pero en pedo
Detectando conceptos en archivo: 1 - AyED - cpp según yo
Detectando conceptos en archivo: 1 - AyED - Unidad3 (7929)
Detectando conceptos en archivo: 1 - AyED - Unidad4 (7930)
Detectando conceptos en archivo: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)
Detectando conceptos en archivo: 1 - MD - Matemáticas discretas by Ramóne Espinosa Armenta (z-lib.org)
Detectando conceptos en archivo: 1 - SyO - 01 Evolucion de las estructuras
Detectando conceptos en archivo: 1 - SyO - 02 Gestion por procesos UNCuyo
Detectando conceptos en archivo: 1 - SyO - 03 Gestion por procesos, indicaroes y estandares para unidades de informacion - Cap 1 y 2
Detectando conceptos en archivo: 1 - SyO - 04 gestion-por-procesos
Detectando conceptos en archivo: 1 - SyO - 05 Arquitectura_empresarial_que_es_y_para_q
Detectando conceptos en archivo: 1 - SyO - 1)

En el diccionario related_table está contenido cuántas veces aparecieron ciertos tokens en la ventana.

A continuación, en vocabulary, se almacenarán solo los que superen una frecuencia mínima y no otra máxima.

In [60]:
min_freq = 50
max_freq = 100

vocabulary = []

for tokens, freq in related_table.items():
    if (freq >= min_freq and freq <= max_freq):
        vocabulary.append(tokens)

concepts_file = "./data/vocabulary.txt"

with open(concepts_file, "wb") as cf:
    for concept in vocabulary:
        cf.write((str(concept) + "\n").encode("utf-8"))

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:2726


En este punto, ya hemos seleccionado conjuntos de tokens que suelen aparecer cerca.

Estos conjuntos serán los conceptos, y pasarán a formar nuestro vocabulario.

Ahora, se debe tokenizar nuevamente los textos planos, utilizando los conceptos.

Para esto, se recorrerá cada archivo de /data/plain mediante una ventana deslizante del mismo tamaño utilizado para detectar conceptos, separando en palabras siempre y cuando no se encuentre dentro de la ventana las palabras de un concepto.

Se generarán nuevos tokens, siendo estos numéricos (/data/tokens_num). Los -1 implican que no se detectó un concepto, mientras que los positivos (o 0) corresponden al índice de un concepto en el vocabulario.

Si en una ventana se detectara más de un concepto, se agregarán todos los que se encuentre. Debido al procesamiento que se realizará más adelante, no debería importar el orden.

In [3]:
import ast

concepts_file = "./data/vocabulary.txt"
vocabulary = []

with open(concepts_file, "rb") as cf:
    lines = cf.read().decode("utf-8").split("\n")[:-1]
    vocabulary = [ast.literal_eval(l) for l in lines]

print("\033[92mTamaño del vocabulario:" + str(len(vocabulary)) + "\033[0m")

Tamaño del vocabulario:2726


In [ ]:
import math

tokens_conceptos_path = "./data/tokens_conceptos"

metrics_2 = {}

window_size_concept_tokenization = window_size * 3

for f in raw_files:
    print("\033[Tokenizando archivo por conceptos: " + f + "\033[0m")
    metrics_2[f] = {}
    found_concepts = 0

    recent_concepts = {}

    with open(plain_path + "/" + f + ".txt", "rb") as pf:
        txt = pf.read().decode("utf-8")
        tokens = [t.text for t in esp.tokenizer(txt)]

        metrics_2[f]["tokens"] = len(tokens)

        with open(tokens_conceptos_path + "/" + f + ".txt", "wb") as tnf:

            unks = 0

            for i in range(len(tokens) - window_size_concept_tokenization):
                window = tokens[i:i+window_size_concept_tokenization]
                for k, v in recent_concepts.items():
                    if v > 0:
                        recent_concepts[k] -= 1

                unks += 1
            
                for ix, concept in enumerate(vocabulary):
                    if all(word in window for word in concept) and (not ix in recent_concepts or recent_concepts[ix] == 0):
                        tnf.write(("-" + str((unks-1)) + " " + str(ix) + " ").encode("utf-8"))
                        unks = 0
                        recent_concepts[ix] = window_size_concept_tokenization
                        found_concepts += 1

                    
    metrics_2[f]["concepts"] = found_concepts

okenizando archivo por conceptos: 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321
okenizando archivo por conceptos: 1 - AyED - cpp según yo pero en pedo
okenizando archivo por conceptos: 1 - AyED - cpp según yo
okenizando archivo por conceptos: 1 - AyED - Unidad3 (7929)
okenizando archivo por conceptos: 1 - AyED - Unidad4 (7930)
okenizando archivo por conceptos: 1 - AyED - Unidades 1 y 2 (cód. fotoc. 7928)


KeyboardInterrupt: 

In [ ]:
print("\033[92m")
for archivo, item in metrics_2.items():
    print("Archivo " + archivo + ":")
    for nombre, valor in item.items():
        print("\t" + nombre + ": " + str(valor))
print("\033[0m")


Archivo 1 - AC - LibroArquitecturadeComputadorasSantiagoPerez090321:
	tokens: 83155



A partir de los tokens numéricos, se iterará por cada secuencia de token con un nuevo tamaño de ventana, mayor, tratando de distinguir conceptos relacionados.

Esta ventana se centrará en cada token (no -1), almacenando en un diccionario el token central, los tokens en el contexto y ejemplos negativos (para evitar que la red neuronal, al entrenar, aprenda que todos los tokens siempre están relacionados).

Hecho esto, podemos finalmente armar nuestro dataset. El mismo retornará (mediante get_item()) un centro, su contexto y sus ejemplos negativos.

A partir de este Dataset, a su vez, se generará un DataLoader.

In [ ]:
import torch

A continuación, se armará la estructura de la red neuronal mediante skipgram, utilizando capas Embedding de pytorch.

Como función de pérdida, se utilizará entropía cruzada binaria (Sigmoidea). Esto es así pues requerimos clasificar dos conceptos según si están o no relacionado.

Se optó por abarcar todo el entrenamiento en una misma función. La misma incluye la inicialización de variables y el ciclo de entrenamiento en sí.

Se generó una función auxiliar para el entrenamiento por medio de GPU, en caso de estar disponible.

In [ ]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

Finalmente, se realizó el entrenamiento:

Para verificar la funcionalidad final que buscamos en el proyecto, se planteó la siguiente función:

Algunos ejemplos de la misma serían: